In [61]:
!pip install qiskit
!pip install qiskit-aer

In [78]:
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import RXGate, RZGate, XGate, IGate, SXGate, CXGate
import numpy as np
from qiskit_aer import Aer

Part 1: Noise Model

In [62]:



def NoiseModel(alpha, beta, qc):
    # Save the registers
    noisy_qc = QuantumCircuit(*qc.qregs, *qc.cregs)

    pauli_gates = ['x', 'y', 'z']

    # Iterate over each instruction in the circuit
    for inst, qargs, cargs in qc.data:
        # Append the original instruction with the correct qubit and clbit references
        noisy_qc.append(inst, qargs, cargs)

        # Handles null case
        if inst.name == 'measure':
            continue

        # Makes the ifs look nicer
        num_qubits = len(qargs)

        # For single-qubit gates
        if num_qubits == 1:
            if np.random.rand() < alpha:
                #Random Pauli
                pauli_gate = np.random.choice(pauli_gates)
                qubit = qargs[0]
                getattr(noisy_qc, pauli_gate)(qubit)

        # For two-qubit gates
        elif num_qubits == 2:
            if np.random.rand() < beta:
                #Random Pauli
                for qubit in qargs:
                    pauli_gate = np.random.choice(pauli_gates)
                    getattr(noisy_qc, pauli_gate)(qubit)

    return noisy_qc



In [63]:
#Testing



qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])


alpha = 0.1
beta = 0.2

# Apply the noise model
noisy_qc = NoiseModel(alpha, beta, qc)


print("Original Circuit:")
print(qc.draw())

print("\nNoisy Circuit:")
print(noisy_qc.draw())


Original Circuit:
     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 

Noisy Circuit:
     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 


In [64]:


def decomposeCircuit(qc):
    # Define the target basis gates
    basis_gates = ['cx', 'id', 'rz', 'sx', 'x']

    # Transpile the circuit to decompose it into the target basis gates
    optimized_circuit = transpile(qc, basis_gates=basis_gates, optimization_level=3)

    return optimized_circuit

In [65]:


# Create a sample quantum circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.rz(0.5, 0)
qc.rx(1.0, 1)

print("Original Circuit:")
print(qc.draw())

# Decompose the circuit into the target gate set
optimized_qc = decomposeCircuit(qc)

print("\nDecomposed Circuit:")
print(optimized_qc.draw())


Original Circuit:
     ┌───┐     ┌─────────┐
q_0: ┤ H ├──■──┤ Rz(0.5) ├
     └───┘┌─┴─┐└┬───────┬┘
q_1: ─────┤ X ├─┤ Rx(1) ├─
          └───┘ └───────┘ 

Decomposed Circuit:
     ┌─────────┐┌────┐┌────────────┐                                          »
q_0: ┤ Rz(π/2) ├┤ √X ├┤ Rz(2.0708) ├──■───────────────────────────────────────»
     └─────────┘└────┘└────────────┘┌─┴─┐┌─────────┐┌────┐┌────────────┐┌────┐»
q_1: ───────────────────────────────┤ X ├┤ Rz(π/2) ├┤ √X ├┤ Rz(4.1416) ├┤ √X ├»
                                    └───┘└─────────┘└────┘└────────────┘└────┘»
«                 
«q_0: ────────────
«     ┌──────────┐
«q_1: ┤ Rz(5π/2) ├
«     └──────────┘


In [66]:
def qft(num_qubits):
    qs = [QuantumRegister(num_qubits)]
    circuit = QuantumCircuit(*qs)
    for j in reversed(range(num_qubits)):
        circuit.h(j)
        num_entanglements = max(0, j - max(0, j + 1 - num_qubits))
        for k in reversed(range(j - num_entanglements, j)):
          #chooses rotation from indecies difference
                lam = np.pi * (2.0 ** (k - j))
                circuit.cp(lam, j, k)
    return circuit

In [76]:


def integer_adder_circuit(a: int, b: int) -> QuantumCircuit:

    #basically the qiskit implementation
    # Step 1: Determine the number of qubits needed
    max_int = max(a, b)
    num_bits = max_int.bit_length() + 1  # +1 for potential overflow

    # Create quantum registers
    qr_a = QuantumRegister(num_bits, name='a')
    qr_b = QuantumRegister(num_bits, name='b')
    cr = ClassicalRegister(num_bits, name='result')

    # Initialize the circuit
    qc = QuantumCircuit(qr_a, qr_b, cr, name='AdderCircuit')

    # Step 2: Encodes the numbers into registers
    a_bin = format(a, f'0{num_bits}b')
    b_bin = format(b, f'0{num_bits}b')

    for i, bit in enumerate(reversed(a_bin)):
        if bit == '1':
            qc.x(qr_a[i])

    for i, bit in enumerate(reversed(b_bin)):
        if bit == '1':
            qc.x(qr_b[i])

    # Step 3: Apply the QFT to the 'b' register
    qc.append(qft(num_bits).to_gate(), qr_b)

    # Step 4: Perform controlled phase rotations
    for i in range(num_bits):
        for j in range(num_bits - i):
            angle = np.pi / (2 ** j)
            qc.cp(angle, qr_a[i], qr_b[i + j])

    # Step 5: Apply the inverse QFT to the 'b' register
    qc.append(qft(num_bits).inverse().to_gate(), qr_b)

    # Step 6: Measure the 'b' register
    qc.measure(qr_b, cr)

    return qc

#testing
if __name__ == "__main__":
    a = 10
    b = 10
    qc = integer_adder_circuit(a, b)


    simulator = Aer.get_backend('qasm_simulator')

    qc_compiled = transpile(qc, simulator)
    job = simulator.run(qc_compiled, shots=1)
    result = job.result()
    counts = result.get_counts(qc_compiled)

    # Get the sum from the binary
    measured_sum_bin = max(counts, key=counts.get)
    measured_sum = int(measured_sum_bin, 2)

    print(f"The sum of {a} and {b} is {measured_sum}.")


The sum of 10 and 10 is 20.


In [87]:
qc = integer_adder_circuit(a, b)
qc2 = decomposeCircuit(qc)
qc3 = NoiseModel(.9,.7,qc)

simulator = Aer.get_backend('qasm_simulator')
qc_compiled = transpile(qc3, simulator)
job = simulator.run(qc_compiled, shots=1)
result = job.result()
counts = result.get_counts(qc_compiled)

# Get the sum from the binary
measured_sum_bin = max(counts, key=counts.get)
measured_sum = int(measured_sum_bin, 2)
print(f"The sum of {a} and {b} is {measured_sum}.")


The sum of 10 and 10 is 20.


I could not get the noise model to alter the result but, it should change the final value for the addition. A lower gate depth would help keep the error from this low. Addtionally, multiple runs would help weed out errors. Higher gate depths increase the probability of a error occuring.